# Heavy-light декомпозиция

HL-декомпозиция — это мощный метод решения задач на запросы на путях, когда также есть запросы обновлений. Если запросов обновления нет, то лучше написать [что-нибудь попроще](http://sereja.me/a/centroid).

<img width='400px' src='http://i38.tinypic.com/6yjmvb.jpg'>

*<center>TODO: найти менее уродливую иллюстрацию</center>*

Heavy-light декомпозицией корневого дерева называется результат следующего процесса: каждой вершины $v$ посмотрим на всех её непосредственных детей $u$, выберем среди них ребёнка $u_{max}$ (с самым большим размером поддерева) и назовём ребро $(v, u)$ *тяжелым* (heavy), а все остальные рёбра — *лёгкими* (light). При этом, если самых «тяжелых» детей будет больше одного, то выберем любого.

Таким образом, дерево распалось на тяжелые и лёгкие рёбра. Докажем несколько полезных свойств такого разбиения.

**Утверждение.** Дерево разбивается на непересекающиеся пути из тяжелых рёбер.

**Доказательство.** В каждую вершину входит не более одного тяжелого ребра, и из каждой вершины исходит не более одного тяжелого ребра.

Назовём *блоком* либо лёгкое ребро, либо вертикальный путь из тяжелых рёбер.

**Утверждение.** На любом вертикальном пути будет не более $O(\log n)$ блоков.

**Доказательство** разбивается на две части:

* Лёгких ребер на вертикальном пути будет не более $O(\log n)$: рассмотрим самую нижнюю вершину и будем идти по вертикальному пути снизу вверх. Каждый раз, когда мы переходим по лёгкому ребру, размер поддерева текущей вершины увеличивается в два раза, потому что если вершина связана с родителем лёгким ребром, то у него есть какой-то другой ребёнок, который не легче текущего.
* Непрерывных путей из тяжелых рёбер будет не более $O(\log n$: если это не конец или начало пути, то каждый такой путь окружают два лёгких ребра, а их всего $O(\log n)$.

**Следствие.** На любом пути будет не более $O(\log n)$ блоков.

Ради этого мы всё и делали: теперь построим какую-нибудь структуру на каждом тяжелого пути (например, дерево отрезков), а при ответе на запрос (скажем, суммы на пути) разобъём его на $O(\log n)$ запросов либо к подотрезкам тяжелых путей, либо к лёгким рёбрам.

## Реализация

Воспользуемся следующим трюком, который нам сильно упростит реализацию. Перенумеруем все вершины так, чтобы их отрезки были в одном большом массиве.

In [ ]:
vector<int> g[maxn];
int s[maxn];

void sizes (int v = 0) {
    s[v] = 1;
    for (int &u : g[v]) {
        sizes(u);
        s[v] += s[u];
        if (s[u] > s[g[v][0]])
            // &u -- это ссылка, и её легально использовать при swap-е
            swap(u, g[v][0]);
    }
}

void hld (int v = 0) {
    in[v] = t++;
    rin[in[v]] = v;
    for (int u : g[v]) {
        nxt[u] = (u == g[v][0] ? nxt[v] : u);
        dfs_hld(u);
    }
    out[v] = t;
}

## Как им решать задачи

Простейший пример задачи на HLD:

> Дано дерево, каждой вершине которого приписано какое-то число. Поступают запросы двух типов:
1. Узнать максимальное число на пути между $v_i$ и $u_i$.
2. Изменить число у $v_i$-той вершины на $x_i$.